In [2]:
import os
import MDAnalysis as mda
from numpy.testing import assert_allclose
import numpy as np

/home/etienne-reboul/anaconda3/envs/Menger_Curvature/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from menger.data import files
from menger.tests.utils import retrieve_results,make_universe,make_filename

In [4]:
from menger.analysis.mengercurvature import MengerCurvature

In [ ]:
md_name="tubulin_chain_a"
u=make_universe(
    topology_name=f"{md_name}.pdb",
    trajectory_name=f"{md_name}.dcd",
)

/home/etienne-reboul/anaconda3/envs/Menger_Curvature/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [5]:
path_topo= "../../data/topology_Tau_minimized_all_atoms_sequence_WT_renumerote.psf"
path_traj="../../data/trajectory_Tau_minimized_all_atoms_sequence_WT.dcd"
u_tau=mda.Universe(path_topo,path_traj)

/home/etienne-reboul/anaconda3/envs/Menger_Curvature/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [6]:
u_tau.atoms.n_atoms

6416

In [7]:
ca_atoms=u_tau.select_atoms("name CA")

In [8]:
len(ca_atoms)

441

In [12]:
tau_analyser=MengerCurvature(u_tau,select="name CA",spacing=2)

In [13]:
tau_analyser.run()

In [15]:
tau_analyser.run_parallel()

In [14]:
tau_analyser.results.curvature_array.shape

(20000, 437)

In [24]:
n_atoms = u.atoms.n_atoms
print(f"Total number of atoms in the universe: {n_atoms}")

Total number of atoms in the universe: 20781


In [27]:
ca_atoms = u.select_atoms("name CA and chainID C")
print(f"Number of CA atoms selected: {len(ca_atoms)}")

Number of CA atoms selected: 450


In [28]:
array_path ="../menger/data/test_data/tubulin_chain_abc_spacing_5_chain_a_curvature_array.npy"
curvature_array = np.load(array_path)

In [29]:
curvature_array.shape

(200, 440)

In [11]:
menger_analyser=MengerCurvature(u,selection="name CA and chainID A",spacing=2)
menger_analyser.run()

In [12]:
menger_analyser.results.keys()

KeysView({'curvature_array': array([[0.8999242 , 0.687924  , 0.737674  , ..., 0.2712029 , 0.22121999,
        0.26541558],
       [0.9213053 , 0.75381017, 0.68575907, ..., 0.3359868 , 0.2606454 ,
        0.37410858],
       [0.7684455 , 0.78443515, 0.73556346, ..., 0.2722758 , 0.20534956,
        0.2685608 ],
       ...,
       [0.9846037 , 0.7679657 , 0.7203345 , ..., 0.26518935, 0.1758656 ,
        0.2585922 ],
       [0.89092684, 0.68492424, 0.7075504 , ..., 0.22315398, 0.22829959,
        0.26677316],
       [0.8631576 , 0.8196901 , 0.7403954 , ..., 0.26505134, 0.20197956,
        0.26506937]], dtype=float32), 'local_curvatures': array([0.88876164, 0.7158077 , 0.62073916, ..., 0.22791246, 0.22153634,
       0.2276711 ], dtype=float32), 'local_flexibilities': array([0.0494775 , 0.05085251, 0.15659133, ..., 0.08930599, 0.04687303,
       0.05459343], dtype=float32)})

In [13]:
u=make_universe(
    topology_name=f"{md_name}.pdb",
    trajectory_name=f"{md_name}.dcd",
)

In [14]:
chainid = set(u.atoms.chainIDs)
print(f"Chain IDs in the universe: {chainid}")

Chain IDs in the universe: {'A'}


In [15]:
template_dict = {
    "md_name": md_name,
    "chainid": "A",
    "select" : "name CA",

}

In [16]:
params_list = []

In [17]:
output_dir = "../menger/data/test_data"

# Test different spacings and chain IDs
for spacing in range(1,6):
    # Test with all chain IDs
    for chain in chainid:
        params_dict = {
            'topology_name': f'{md_name}.pdb',
            'trajectory_name': f'{md_name}.dcd', 
            'select': f"name CA and chainID {chain}",
            'spacing': spacing,
            'md_name': md_name,
            'chainid': chain.lower(),
            "rtol": 1e-04
        }
        params_list.append(params_dict)
        menger_analyser = MengerCurvature(u, f"name CA and chainID {chain}", spacing=spacing)
        menger_analyser.run()
        results = menger_analyser.results

        # Define test cases
        test_cases = [
            ('curvature_array', results.curvature_array),
            ('local_curvatures', results.local_curvatures),
            ('local_flexibilities', results.local_flexibilities)
        ]
        # write results to file
        # Save full menger array
        for attr_name, test_value in test_cases:

            filename = make_filename(md_name, spacing=spacing, chainid=chain.lower(), metric=attr_name)
   
            np.save(os.path.join(output_dir, filename), test_value)     
    



    

In [18]:
params_list

[{'topology_name': 'tubulin_chain_a.pdb',
  'trajectory_name': 'tubulin_chain_a.dcd',
  'select': 'name CA and chainID A',
  'spacing': 1,
  'md_name': 'tubulin_chain_a',
  'chainid': 'a',
  'rtol': 0.0001},
 {'topology_name': 'tubulin_chain_a.pdb',
  'trajectory_name': 'tubulin_chain_a.dcd',
  'select': 'name CA and chainID A',
  'spacing': 2,
  'md_name': 'tubulin_chain_a',
  'chainid': 'a',
  'rtol': 0.0001},
 {'topology_name': 'tubulin_chain_a.pdb',
  'trajectory_name': 'tubulin_chain_a.dcd',
  'select': 'name CA and chainID A',
  'spacing': 3,
  'md_name': 'tubulin_chain_a',
  'chainid': 'a',
  'rtol': 0.0001},
 {'topology_name': 'tubulin_chain_a.pdb',
  'trajectory_name': 'tubulin_chain_a.dcd',
  'select': 'name CA and chainID A',
  'spacing': 4,
  'md_name': 'tubulin_chain_a',
  'chainid': 'a',
  'rtol': 0.0001},
 {'topology_name': 'tubulin_chain_a.pdb',
  'trajectory_name': 'tubulin_chain_a.dcd',
  'select': 'name CA and chainID A',
  'spacing': 5,
  'md_name': 'tubulin_chain_a

In [19]:
md_name="tubulin_chain_abc"
u=make_universe(
    topology_name=f"{md_name}.pdb",
    trajectory_name=f"{md_name}.dcd",
)

In [20]:
chainid = set(u.atoms.chainIDs)
print(f"Chain IDs in the universe: {chainid}")

Chain IDs in the universe: {'A', 'C', 'B'}


In [21]:
output_dir = "../menger/data/test_data"

# Test different spacings and chain IDs
for spacing in range(1,6):
    # Test with all chain IDs
    for chain in chainid:
        params_dict = {
            'topology_name': f'{md_name}.pdb',
            'trajectory_name': f'{md_name}.dcd', 
            'select': f"name CA and chainID {chain}",
            'spacing': spacing,
            'md_name': md_name,
            'chainid': chain.lower(),
            "rtol": 1e-04
        }
        params_list.append(params_dict)
        menger_analyser = MengerCurvature(u, f"name CA and chainID {chain}", spacing=spacing)
        menger_analyser.run()
        results = menger_analyser.results

        # Define test cases
        test_cases = [
            ('curvature_array', results.curvature_array),
            ('local_curvatures', results.local_curvatures),
            ('local_flexibilities', results.local_flexibilities)
        ]
        # write results to file
        # Save full menger array
        for attr_name, test_value in test_cases:

            filename = make_filename(md_name, spacing=spacing, chainid=chain.lower(), metric=attr_name)
   
            np.save(os.path.join(output_dir, filename), test_value)     
    
 

In [22]:
import json
import os

# Define the output path
json_path = os.path.join("../menger/data", "MengerCurvature_test_parameters.json")

# Write the params_list to JSON file
with open(json_path, 'w') as f:
    json.dump(params_list, f, indent=4)

In [ ]:
chainid 

In [ ]:
menger_analyser = MengerCurvature(u,"name CA and chainID A")
menger_analyser.run()


In [ ]:
%%timeit
menger_analyser = MengerCurvature(u,"name CA")
menger_analyser.run()

In [ ]:
local_curvatures = menger_analyser.results.local_curvatures
local_flexibilities = menger_analyser.results.local_flexibilities

In [ ]:
def make_filename(md_name : str, spacing : int , metric : str,chainid : str| None = None) -> str:

    filename : str = ""
    if chainid is None:
        filename=f"{md_name}_{metric}.npy"
    else:
       filename= f"{md_name}_spacing_{spacing}_chain_{chainid}_{metric}.npy"

    return filename

In [ ]:
make_filename(md_name,spacing=1,metric="local_curvatures",chainid=chainid)

In [ ]:
import os
import numpy as np
from menger.analysis.mengercurvature import MengerCurvature
from pathlib import Path

# Create test directory if it doesn't exist
output_dir = "../menger/data/test_data"
os.makedirs(output_dir, exist_ok=True)

results = []
spacings = range(1,6)

for i, spacing in enumerate(spacings):

    # run MengerCurvature analysis
    menger_analyser = MengerCurvature(u, "name CA", spacing=spacing)
    menger_analyser.run()# Get results for current spacing
    
    # Save local curvatures
    output_path = os.path.join(output_dir,make_filename(md_name=md_name,
                                                        spacing=spacing,
                                                        metric="local_curvatures",
                                                        chainid=chainid))

    np.save(output_path, menger_analyser.results.local_curvatures)

    # Save local flexibilities
    output_path = os.path.join(output_dir, make_filename(md_name=md_name,
                                                        spacing=spacing,
                                                        metric="local_flexibilities",
                                                        chainid=chainid))
    np.save(output_path, menger_analyser.results.local_flexibilities)
    
    # Save full menger array
    output_path = os.path.join(output_dir, make_filename(md_name=md_name,
                                                        spacing=spacing,
                                                        metric="curvature_array",
                                                        chainid=chainid))
    np.save(output_path, menger_analyser.results.curvature_array)


In [ ]:
output_dir

In [ ]:
def test_menger_curvature(md_name  : str,universe: np.ndarray,select : str, spacing: int):
    menger_analyser = MengerCurvature(universe, select, spacing)
    menger_analyser.run()
    results = menger_analyser.results
    
    # test menger array 
    test_curvature_array = results.curvature_array
    expected_curvature_array = retrieve_results(md_name, spacing, "curvature_array")
    assert_allclose(test_curvature_array,
                    expected_curvature_array,
                    err_msg="Menger array is not as expected" + \
                    f" for md: {md_name} spacing {spacing}"
                    )

    # test local curvatures
    test_local_curvatures = results.local_curvatures
    expected_local_curvatures = retrieve_results(md_name, spacing, "local_curvatures")
    assert_allclose(test_local_curvatures,
                    expected_local_curvatures,
                    err_msg="Local curvatures are not as expected"+ \
                    f" for md: {md_name} spacing {spacing}"
                    )

    # test local flexibilities
    test_local_flexibilities = results.local_flexibilities
    expected_local_flexibilities = retrieve_results(md_name, spacing, "local_flexibilities")
    assert_allclose(test_local_flexibilities,
                    expected_local_flexibilities,
                    err_msg="Local flexibilities are not as expected"+ \
                    f" for md: {md_name} spacing {spacing}"
                        )

In [ ]:
spacing = 2

In [ ]:
menger_analyser = MengerCurvature(u, "name CA", spacing)
menger_analyser.run()
results = menger_analyser.results

In [ ]:
# md_name = "tubulin_chain_a"
expected_curvature_array = retrieve_results(md_name, spacing, "curvature_array")

In [ ]:
files.TEST_DATA_DIR

In [ ]:
expected_curvature_array

In [ ]:
test_curvature_array = results.curvature_array

In [ ]:
test_curvature_array

In [ ]:

# test menger array 
test_curvature_array = results.curvature_array

assert_allclose(test_curvature_array,
                expected_curvature_array,
                err_msg="Menger array is not as expected" + \
                f" for md: {md_name} spacing {spacing}"
                )


In [ ]:
! ls /home/etienne-reboul/anaconda3/envs/Menger_Curvature/lib/python3.10/site-packages/menger/data/test_data/

In [ ]:
from menger.analysis.mengercurvature import menger_curvature

In [ ]:
frame= np.array([[13.31, 34.22, 34.36],
                                [16.89, 33.47, 35.28],
                                [20.4 , 34.65, 34.76],
                                [23.99, 33.21, 34.96],
                                [27.52, 34.44, 34.73],
                                [31.27, 33.34, 35.16],
                                [34.95, 34.55, 34.84],
                                [38.57, 33.49, 35.07],
                                [42.11, 34.67, 34.64],
                                [45.72, 33.37, 34.84],
                                [49.49, 34.3 , 34.62],
                                [53.24, 33.33, 34.85],
                                [56.58, 35.18, 34.74]],
                                dtype=np.float32)

In [ ]:
np.round(menger_curvature(frame, 2), 2)